# Find Missing Parents

I'll give two approaches:

1. Carrying the association between ID and node as we go
2. Precomputing a global map from ID to name, and just working with sets of IDs.


## Setup
This just creates test data.

In [1]:
# set up simple environment
# start with suffixes.
suffixes = ["a", "b", "c", "d", "e", "f", "g"]

def setup():
    '''Set up our data.'''
    # Produce the names as a generator
    parent_names =  ("parent_"+p for p in suffixes)
    # Do the same for the children.
    child_names = ("child_" + str(i) for i in range(100, 120))
    # Don't create these; they are what we want to find
    missing = {"parent_c", "parent_e", "parent_f"}
    # Create the parent nodes, except for the missing ones above.
    parent_nodes = {
        n:{"name": n, "id": i, "parent": i}
        for (i, n) in enumerate(parent_names, 1)
        if not n in missing
        }
    # Only use parents in the ID range 1 - 4, so only "parent_c" will be
    # missing-but-used
    child_nodes = {
        n:{"name":n, "id": i, "parent": (i % 4) + 1}
        for (i, n) in enumerate(child_names, 100)
    }
    # Combine the two sets of nodes.
    # Python 3 way:
    # nodes = {**parent_nodes, **child_nodes}
    # Python 2.7 way:
    nodes = {}
    nodes.update(parent_nodes)
    nodes.update(child_nodes)
    return nodes

def create_parent(node):
    '''Dummy create_parent
    
* node: the child node (or child node name) to create a parent for'''
    print('create_parent', node)
    if isinstance(node, str):
        return node
    else:
        return node["name"]

## Our test data:
OK, now we have our test data set up. Let's look at it.

Note that no child has a parent of 3, 5, 6, or 7

In [2]:
nodes = setup()
nodes

{'parent_a': {'name': 'parent_a', 'id': 1, 'parent': 1},
 'parent_b': {'name': 'parent_b', 'id': 2, 'parent': 2},
 'parent_d': {'name': 'parent_d', 'id': 4, 'parent': 4},
 'parent_g': {'name': 'parent_g', 'id': 7, 'parent': 7},
 'child_100': {'name': 'child_100', 'id': 100, 'parent': 1},
 'child_101': {'name': 'child_101', 'id': 101, 'parent': 2},
 'child_102': {'name': 'child_102', 'id': 102, 'parent': 3},
 'child_103': {'name': 'child_103', 'id': 103, 'parent': 4},
 'child_104': {'name': 'child_104', 'id': 104, 'parent': 1},
 'child_105': {'name': 'child_105', 'id': 105, 'parent': 2},
 'child_106': {'name': 'child_106', 'id': 106, 'parent': 3},
 'child_107': {'name': 'child_107', 'id': 107, 'parent': 4},
 'child_108': {'name': 'child_108', 'id': 108, 'parent': 1},
 'child_109': {'name': 'child_109', 'id': 109, 'parent': 2},
 'child_110': {'name': 'child_110', 'id': 110, 'parent': 3},
 'child_111': {'name': 'child_111', 'id': 111, 'parent': 4},
 'child_112': {'name': 'child_112', 'id'

## Find the missing parents.

Because we don't have an easy lookup from ID to node or name, we'll associate each ID a node in a map. Otherwise, we'd use sets rather than maps.

### Find all the parents.


In [3]:
parents = {node["id"]:node for node in nodes.values() if node["id"] == node["parent"]}
parents

{1: {'name': 'parent_a', 'id': 1, 'parent': 1},
 2: {'name': 'parent_b', 'id': 2, 'parent': 2},
 4: {'name': 'parent_d', 'id': 4, 'parent': 4},
 7: {'name': 'parent_g', 'id': 7, 'parent': 7}}

Now let's find the children the same way.

### Find all the children.

In [4]:
children = {node["id"]:node for node in nodes.values() if node["id"] != node["parent"]}
children

{100: {'name': 'child_100', 'id': 100, 'parent': 1},
 101: {'name': 'child_101', 'id': 101, 'parent': 2},
 102: {'name': 'child_102', 'id': 102, 'parent': 3},
 103: {'name': 'child_103', 'id': 103, 'parent': 4},
 104: {'name': 'child_104', 'id': 104, 'parent': 1},
 105: {'name': 'child_105', 'id': 105, 'parent': 2},
 106: {'name': 'child_106', 'id': 106, 'parent': 3},
 107: {'name': 'child_107', 'id': 107, 'parent': 4},
 108: {'name': 'child_108', 'id': 108, 'parent': 1},
 109: {'name': 'child_109', 'id': 109, 'parent': 2},
 110: {'name': 'child_110', 'id': 110, 'parent': 3},
 111: {'name': 'child_111', 'id': 111, 'parent': 4},
 112: {'name': 'child_112', 'id': 112, 'parent': 1},
 113: {'name': 'child_113', 'id': 113, 'parent': 2},
 114: {'name': 'child_114', 'id': 114, 'parent': 3},
 115: {'name': 'child_115', 'id': 115, 'parent': 4},
 116: {'name': 'child_116', 'id': 116, 'parent': 1},
 117: {'name': 'child_117', 'id': 117, 'parent': 2},
 118: {'name': 'child_118', 'id': 118, 'parent

So the question now is, what parents are in use? Let's construct that set. Again, we'll use a map rather than a set, so we remember an example child to go with each `in_use` parent.

In [5]:
in_use = {node["parent"]: node for node in children.values()}
in_use

{1: {'name': 'child_116', 'id': 116, 'parent': 1},
 2: {'name': 'child_117', 'id': 117, 'parent': 2},
 3: {'name': 'child_118', 'id': 118, 'parent': 3},
 4: {'name': 'child_119', 'id': 119, 'parent': 4}}

So the next question is, which of these parents don't exist?

Here, we'll use set operations, removing the existing parents from the ones in use.

In [6]:
need_to_create = set(in_use.keys()).difference(parents.keys())
need_to_create

{3}

But we want child nodes, not just parent ID's, so we'll refer back to our `in_use` variable to find those.

In [7]:
to_create_parents = [in_use[n] for n in need_to_create]
to_create_parents

[{'name': 'child_118', 'id': 118, 'parent': 3}]

And those are the ones (only 1 in this case) that we need to create a parent for!

In [8]:
[create_parent(node) for node in to_create_parents]

create_parent {'name': 'child_118', 'id': 118, 'parent': 3}


['child_118']

## Recap

So to review what we did:

```python
parents = {node["id"]:node for node in nodes.values() if node["id"] == node["parent"]}  #1
children = {node["id"]:node for node in nodes.values() if node["id"] != node["parent"]} #2
in_use = {node["parent"]: node for node in children.values()}                           #3
need_to_create = set(in_use.keys()).difference(parents.keys())                          #4
to_create_parents = (in_use[n] for n in need_to_create)                                 #5
[create_parent(node) for node in to_create_parents]                                     #6
```

Or:
1. Find the parents, in a dict indexed by `id`.
2. Find the children, in a dict indexed by `id`.
3. Find the parents in use by scanning the children, in a dict indexed by parent.
4. Find the parents in use but not existing, but removing the ones from #3 that exist.
5. Find the selected child for each missing parent.
6. Create the parents.

### Done!


## A Second Approach

First, we'll set up our data again.

In [9]:
nodes = setup()
nodes

{'parent_a': {'name': 'parent_a', 'id': 1, 'parent': 1},
 'parent_b': {'name': 'parent_b', 'id': 2, 'parent': 2},
 'parent_d': {'name': 'parent_d', 'id': 4, 'parent': 4},
 'parent_g': {'name': 'parent_g', 'id': 7, 'parent': 7},
 'child_100': {'name': 'child_100', 'id': 100, 'parent': 1},
 'child_101': {'name': 'child_101', 'id': 101, 'parent': 2},
 'child_102': {'name': 'child_102', 'id': 102, 'parent': 3},
 'child_103': {'name': 'child_103', 'id': 103, 'parent': 4},
 'child_104': {'name': 'child_104', 'id': 104, 'parent': 1},
 'child_105': {'name': 'child_105', 'id': 105, 'parent': 2},
 'child_106': {'name': 'child_106', 'id': 106, 'parent': 3},
 'child_107': {'name': 'child_107', 'id': 107, 'parent': 4},
 'child_108': {'name': 'child_108', 'id': 108, 'parent': 1},
 'child_109': {'name': 'child_109', 'id': 109, 'parent': 2},
 'child_110': {'name': 'child_110', 'id': 110, 'parent': 3},
 'child_111': {'name': 'child_111', 'id': 111, 'parent': 4},
 'child_112': {'name': 'child_112', 'id'

We'll go through similar steps as in the first approach. But before we begin, we make a map from `id` to `(name, id, parentId)` so we can just work with IDs.

There are a couple of advantages to this approach:

* We don't need to keep any actual nodes around between the initial pass and the final creation step. This ccould save on memory, in theory. (I doubt this is significant in practice here.)
* It's a little easier to see what's going on.

The downside is a bit of work up front to collect the information.

First, we'll collect our ids and get a map to `(name, id, parentId)`

In [10]:
ids = {node["id"]:(name,node["id"],node["parent"]) for (name,node) in nodes.items()}
ids

{1: ('parent_a', 1, 1),
 2: ('parent_b', 2, 2),
 4: ('parent_d', 4, 4),
 7: ('parent_g', 7, 7),
 100: ('child_100', 100, 1),
 101: ('child_101', 101, 2),
 102: ('child_102', 102, 3),
 103: ('child_103', 103, 4),
 104: ('child_104', 104, 1),
 105: ('child_105', 105, 2),
 106: ('child_106', 106, 3),
 107: ('child_107', 107, 4),
 108: ('child_108', 108, 1),
 109: ('child_109', 109, 2),
 110: ('child_110', 110, 3),
 111: ('child_111', 111, 4),
 112: ('child_112', 112, 1),
 113: ('child_113', 113, 2),
 114: ('child_114', 114, 3),
 115: ('child_115', 115, 4),
 116: ('child_116', 116, 1),
 117: ('child_117', 117, 2),
 118: ('child_118', 118, 3),
 119: ('child_119', 119, 4)}

Then we'll augment that with a map from parent to a child.

In [11]:
parent_map = {parent:child for (name, child, parent) in ids.values()}  #2a
parent_map

{1: 116, 2: 117, 4: 119, 7: 7, 3: 118}

So now we do essentially the same as above, but without any maps. Instead, we create sets directly,
using the information in `ids`, and then `parent_map` to find the child to use.

Note that the difference between a map comprehension and a set comprehension is that instead of `key:value`, we just write `value`.

For example, instead of this map:

```python
{"key_" + str(i):i for i in range(0, 10) if i % 3 = 0}
```

We write this set:

```python
{"key_" + str(i) for i in range(0, 10) if i % 3 = 0}
```

The other new thing below is using a generator comprehension to supply values to a set comprehension.

In step #3, we want not the child IDs, but rather the result of looking up `id` as `ids[id]`.
We could do this as two steps with a variable. But here, we're just doing it to be able to destructure.

```python
(ids[id] for id in children)
```

So we don't really need a variable, but if we wanted to, it would look like this:
```python
children_info = (ids[id] for id in children)
in_use = {parent for (id, name, parent) in children_info}
```

We could also write that as:

```python
in_use = {ids[id][2] for id in children}
```

But `ids[id][2]` is hard to understand. Doing the `ids[id]` first lets us use destructuring to access
`parent`. A little longer and more complex, but a bit more explicit about what we're doing. But in step 6,
I use `ids[id][0]`, so you can compare see it both ways.


In [12]:
parents = {id for (name, id, parent) in ids.values() if id == parent}    #1
children = {id for (name, id, parent) in ids.values() if id != parent}   #2
in_use = {parent for (id, name, parent) in (ids[id] for id in children)} #3
need_to_create = in_use.difference(parents)                              #4
orphans = {parent_map[parent] for parent in need_to_create}              #5
[create_parent(ids[id][0]) for id in orphans]                            #6


create_parent child_118


['child_118']

These are very similar. In the first example, we combine the use of maps and sets. In the second, we separate them out, by first indexing our data,
then performing the set operations.

The extra work up front makes things later simpler and easier to understand.

The benefit here is small; if we performed more complex operations, the benefit of pre-indexing becomes much greater.
It's not just that the code is easier to read. It's easier to write, and easier to think about what to write.

But both techniques are valid:

* Passing on all the relevant data at each step.
* Pre-indexing, and just pass keys to look up the data.